# LangChain Quick Start Demo

This notebook demonstrates how to create a simple LangChain agent with tools.


## 1. Install Dependencies


In [ ]:
!pip install "langchain>=1.0.0" "langchain-openai>=1.0.0" "requests" "fastapi" "uvicorn" --target ~/packages

## 2. Import Libraries


In [ ]:
import sys
from pathlib import Path

user_site_packages = Path.home() / "packages"
if str(user_site_packages) not in sys.path:
    sys.path.insert(0, str(user_site_packages))

from langchain.agents import create_agent
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from typing import Dict, Any
import requests
import os

print("Libraries imported")

## 3. Set Configuration

Set your API key and model configuration (optional, can also use environment variables):


In [ ]:
# Set your API key by the OPENAI_API_KEY environment variable
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY", "<Your API Key>")
MODEL_NAME = os.getenv("MODEL_NAME", "deepseek-chat")
BASE_URL = os.getenv("BASE_URL", "https://api.deepseek.com")

print("Environ configured")

## 4. Define a Tool

Create a simple weather tool using the `@tool` decorator:


In [ ]:
@tool
def get_weather(city: str) -> Dict[str, Any]:
    """Get current weather for a city.
    
    Args:
        city: City name (e.g., Beijing, Shanghai, New York)
        
    Returns:
        Dictionary with weather information
    """
    try:
        url = f"https://wttr.in/{city}?format=j1"
        response = requests.get(url, timeout=30)
        response.raise_for_status()
        data = response.json()
        
        current = data["current_condition"][0]
        return {
            "city": city,
            "temperature": f"{current['temp_C']}°C",
            "humidity": f"{current['humidity']}%",
        }
    except Exception as e:
        return {"error": f"Failed to get weather: {str(e)}"}

print("Defined get_weather tool")

## 5. Create LLM Model

Initialize the language model (using DeepSeek in this example):


In [ ]:
llm = ChatOpenAI(
    model=MODEL_NAME,
    base_url=BASE_URL if BASE_URL else None,  # None means use OpenAI default
    temperature=0
)


print("Created LLM instance")

## 6. Create Agent

Create an agent with the weather tool:


In [ ]:
agent = create_agent(
    model=llm,
    tools=[get_weather],
    system_prompt="You are a helpful weather assistant."
)

print("Created Agent")

## 7. Run the Agent

Invoke the agent with a query:


In [ ]:
result = await agent.ainvoke({
    "messages": [
        {"role": "user", "content": "What's the weather in Beijing?"}
    ]
})

print("Invoked")

## 8. Display the Result


In [ ]:
def print_result(result: Dict[str, Any]):
    if isinstance(result, dict) and "messages" in result:
        last_message = result["messages"][-1]
        if hasattr(last_message, "content"):
            print(last_message.content)
        else:
            print(result)
    else:
        print(result)

print_result(result)

## Try Different Queries

You can try asking different questions:


In [ ]:
# Try another query
result2 = await agent.ainvoke({
    "messages": [
        {"role": "user", "content": "What's the weather in Shanghai?"}
    ]
})

print_result(result2)

## 9. FastAPI Service Example

You can also run the agent as a FastAPI web service for production use:


In [ ]:
# Import FastAPI components
from fastapi import FastAPI
from pydantic import BaseModel
from threading import Thread
import time
import requests

# Create a simple FastAPI app
api_app = FastAPI(title="LangChain Agent API")

class ChatRequest(BaseModel):
    message: str

@api_app.post("/chat")
async def chat(request: ChatRequest):
    """ Chat endpoint """
    result = await agent.ainvoke({
        "messages": [
            {"role": "user", "content": request.message}
        ]
    })

    if isinstance(result, dict) and "messages" in result:
        last_message = result["messages"][-1]
        if hasattr(last_message, "content"):
            return {"response": last_message.content}
    return {"response": str(result)}


print("FastAPI app created. Use the next cell to start the server.")


### Start the FastAPI Server

**Note**: In a notebook, you can start the server in a background thread. For production, run it as a separate process.


In [ ]:
# Start server in background thread (for notebook demonstration)

from uvicorn import Config, Server

# Create a server instance that can be controlled
config = Config(api_app, host="127.0.0.1", port=8000, log_level="info")
server = Server(config)

def run_server():
    server.run()

# Use daemon=True so the thread stops automatically when the kernel restarts
# This is safe for notebook demonstrations
# For production, use process managers instead of threads
server_thread = Thread(target=run_server, daemon=True)
server_thread.start()

# Wait a moment for the server to start
time.sleep(2)
print("✓ FastAPI server started at http://127.0.0.1:8000")

### Test the API

Now you can call the API using HTTP requests:


In [ ]:
# Test the API endpoint
response = requests.post(
    "http://127.0.0.1:8000/chat",
    json={"message": "What's the weather in Shanghai?"},
    timeout=30
)

print(f"Status Code: {response.status_code}")
print("Response:")
print(response.json().get('response'))


### Stop the Server

You can stop the server by calling its shutdown method:


In [ ]:
# Stop the server
if 'server' in globals() and server.started:
    server.should_exit = True
    print("✓ Server shutdown requested. It will stop after handling current requests.")
    print("  Note: The server will also stop automatically when you restart the kernel.")
else:
    print("Server is not running or has already stopped.")


## 10. More Resources

For more resources on developing AI Agents with Langchain, see:

- [Langchain Documentation](https://docs.langchain.com/oss/python/langchain/overview) - The official Langchain documentation where all usage-related documentation can be found.
- [Langchain Academy](https://academy.langchain.com/) - The official Langchain Academy provides extensive educational resources. The [Foundation Introduction to Langchain Python](https://academy.langchain.com/courses/foundation-introduction-to-langchain-python) course introduces the fundamentals of developing Agents with Langchain.